In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./functionary-small-v3.2")
model = AutoModelForCausalLM.from_pretrained("./functionary-small-v3.2", trust_remote_code=True).to('cuda')


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    }
                },
                "required": ["location"]
            }
        }
    }
]
messages = [{"role": "user", "content": "What is the weather in Istanbul and Singapore respectively?"}]

final_prompt = tokenizer.apply_chat_template(messages, tools, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(final_prompt, return_tensors="pt").to("cuda")
pred = model.generate_tool_use(**inputs, max_new_tokens=128, tokenizer=tokenizer)
print(tokenizer.decode(pred.cpu()[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are capable of executing available function(s) if required.
Only execute function(s) when absolutely necessary.
Ask for the required input to:recipient==all
Use JSON for function arguments.
Respond in this format:
>>>${recipient}
${content}
Available functions:
// Supported function definitions that should be called when necessary.
namespace functions {

// Get the current weather
type get_current_weather = (_: {
// The city and state, e.g. San Francisco, CA.
location: string,
}) => any;

} // namespace functions<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather in Istanbul and Singapore respectively?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

>>>{
    "role": "assistant",
    "content": null,
    "tool_calls": [
        {
            "name": "get_current_weather",
            "arguments": "{\"location\": \"Istanbul\"}"
        },
        {
            "name": "get_curren